In [ ]:
# -*- coding:utf-8 -*-
#  author: yukun
import requests
from bs4 import BeautifulSoup
 
    
# 发出请求获得HTML源码的函数
def get_html(url):
    # 伪装成浏览器访问
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    resp = requests.get(url, headers=headers).text
 
    return resp
 
# 解析页面，获得数据信息
def html_parse():
    # 调用函数，for循环迭代出所有页面
    for url in all_page():
        # BeautifulSoup的解析
        soup = BeautifulSoup(get_html(url), 'lxml')
        # 书名
        alldiv = soup.find_all('div', class_='pl2')
        names = [a.find('a')['title'] for a in alldiv]
        # 作者
        allp = soup.find_all('p', class_='pl')
        authors = [p.get_text() for p in allp]
        # 评分
        starspan = soup.find_all('span', class_='rating_nums')
        scores = [s.get_text() for s in starspan]
        # 简介
        sumspan = soup.find_all('span', class_='inq')
        sums = [i.get_text() for i in sumspan]
        for name, author, score, sum in zip(names, authors, scores, sums):
            name = '书名：' + str(name) + '\n'
            author = '作者：' + str(author) + '\n'
            score = '评分：' + str(score) + '\n'
            sum = '简介：' + str(sum) + '\n'
            data = name + author + score + sum
            # 保存数据
            f.writelines(data + '=======================' + '\n')
 
# 获得所有页面的函数
def all_page():
    base_url = 'https://book.douban.com/top250?start='
    urllist = []
    # 从0到225，间隔25的数组
    for page in range(0, 250, 25):
        allurl = base_url + str(page)
        urllist.append(allurl)
 
    return  urllist
 
# 文件名
filename = '豆瓣图书Top250.txt'
# 保存文件操作
f = open(filename, 'w', encoding='utf-8')
# 调用函数
html_parse()
f.close()
print('保存成功。')

In [ ]:
# -*- coding:utf-8 -*-
#  author: yukun
import requests
import os
import time
from bs4 import BeautifulSoup
 

# 发出请求获得HTML源码
def get_html(url):
    # 指定一个浏览器头
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    # 代理，免费的代理只能维持一会可能就没用了，自行更换
    proxies = {'http': '111.23.10.27:8080'}
    try:
        # Requests库的get请求
        resp = requests.get(url, headers=headers)
    except:
        # 如果请求被阻，就使用代理
        resp = requests.get(url, headers=headers, proxies=proxies)
        print('代理登陆中')
 
    return resp
 
# 创建文件夹的函数，保存到D盘
def mkdir(path):
    # os.path.exists(name)判断是否存在路径
    # os.path.join(path, name)连接目录与文件名
    isExists = os.path.exists(os.path.join("E:\jiandan", path))
    # 如果不存在
    if not isExists:
        print('makedir', path)
        # 创建文件夹
        os.makedirs(os.path.join("E:\jiandan", path))
        # 切换到创建的文件夹
        os.chdir(os.path.join("E:\jiandan", path))
        return True
    # 如果存在了就返回False
    else:
        print(path, 'already exists')
        return False

# 获得图片地址调用download函数进行下载
def get_imgs():
    # 调用函数获得所有页面
    for url in all_page():
        path = url.split('-')[-1]
        # 创建文件夹的函数
        mkdir(path)
        # 调用请求函数获得HTML源码
        html = get_html(url).text
        # 使用lxml解析器，也可以使用html.parser
        soup = BeautifulSoup(html, 'lxml')
        # css选择器
        #allimgs = soup.select('div.text &gt; p &gt; img')
        allimgs = soup.select('div > div > div.text > p > img')
        #allimgs = soup.select('#comment')
        
        print(allimgs)
        
        # 调用download函数下载保存
        download(allimgs)
    # 执行完毕打出ok
    print('ok')

# 获得所有页面
def all_page():
    base_url = 'http://jandan.net/ooxx/'
    # BeautifulSoup解析页面得到最高页码数
    soup = BeautifulSoup(get_html(base_url).text, 'lxml')
    # 获得最高页码数
    allpage = soup.find('span', class_="current-comment-page").get_text()[1:-1]
    urllist = []
    # for循环迭代出所有页面，得到url
    #for page in range(1, int(allpage)+1):
    for page in range(1, 2):
        allurl = base_url + 'page-' + str(page)
        urllist.append(allurl)
    return urllist
 
# 保存图片函数，传入的参数是一页所有图片url集合
def download(list):
    for img in list:
        urls = img['src']
        # 判断url是否完整
        if urls[0:5] == 'http:':
            img_url = urls
        else:
            img_url = 'http:' + urls
        filename = img_url.split('/')[-1]
        # 保存图片
        with open(filename, 'wb') as f:
            # 直接过滤掉保存失败的图片，不终止程序
            try:
                f.write(get_html(img_url).content)
                print('Sucessful image:',filename)
            except:
                print('Failed:',filename)

if __name__ == '__main__':
    # 计时
    t1 = time.time()
    # 调用函数
    get_imgs()
    print(time.time() - t1)

## 1024

In [ ]:
# -*- coding:utf-8 -*-
#  author: yukun
import requests
import os
import time
import re
from bs4 import BeautifulSoup
 

# 发出请求获得HTML源码
def get_html(url):
    # 指定一个浏览器头
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    # 代理，免费的代理只能维持一会可能就没用了，自行更换
    proxies = {'http': '111.23.10.27:8080'}
    try:
        # Requests库的get请求
        resp = requests.get(url, headers=headers)
    except:
        # 如果请求被阻，就使用代理
        resp = requests.get(url, headers=headers, proxies=proxies)
        print('代理登陆中')
    return resp
 
# 创建文件夹的函数，保存到E盘
def mkdir(path):    
    isExists = os.path.exists(os.path.join("E:\\1024\\街拍", path))# os.path.exists(name)判断是否存在路径
    # 如果不存在
    if not isExists:
        #print('makedir', path)
        # 创建文件夹
        os.makedirs(os.path.join("E:\\1024\\街拍", path))# os.path.join(path, name)连接目录与文件名
        # 切换到创建的文件夹
        os.chdir(os.path.join("E:\\1024\\街拍", path))
        return True
    # 如果存在了就返回False
    else:
        print(path, 'already exists')
        return False
    
# 获得所有页面
def all_page():
    base_url ='http://p3.csgfnmdb.com/pw/thread.php?fid=49'
    get_html(base_url).encoding='utf-8'
    soup = BeautifulSoup(get_html(base_url).text, 'lxml')
    urllist = []
    t1 = soup.find_all('a')
    #print(t1)
    for t2 in t1:
        t3 = t2.get('href') 
        if "html_data" in str(t3):
            urllist.append('http://p3.csgfnmdb.com/pw/'+ str(t3))
    urllist=list(set(urllist))
    #print(urllist)
    
    return urllist
# 保存图片函数，传入的参数是一页所有图片url集合

# 获得图片地址调用download函数进行下载
def get_imgs():
    # 调用函数获得所有页面
    #urllist = all_page()
    for url in all_page():
        new = get_html(url)
        new.encoding='utf-8'
        soup = BeautifulSoup(new.text, 'lxml')
        path = soup.find('title').text.split('|')[0]
        print(path)
    
        # 创建文件夹的函数
        mkdir(path)
        # 调用请求函数获得HTML源码
        html = get_html(url).text
        # 使用lxml解析器，也可以使用html.parser
        soup = BeautifulSoup(html, 'lxml')
        #print(soup)
        # css选择器
        allimgs = soup.select('img')
        #print(allimgs)        
        # 调用download函数下载保存
        download(allimgs)
    # 执行完毕打出ok
    print('ok')




def download(list):
    for img in list:
        urls = img['src']
        # 判断url是否完整
        if urls[0:5] == 'http:':
            img_url = urls
        else:
            img_url = 'http:' + urls
        filename = img_url.split('/')[-1]
        tpye = filename.split('.')[-1]
        if  tpye =='jpg':
            # 保存图片
            with open(filename, 'wb') as f:
                # 直接过滤掉保存失败的图片，不终止程序
                try:
                    f.write(get_html(img_url).content)
                    #print('Sucessful image:',filename)
                except:
                    print('Failed:',filename)
        

if __name__ == '__main__':
    # 计时
    t1 = time.time()
    # 调用函数
    get_imgs()
    print(time.time() - t1)

惊艳的黑色包臀裙雪白美腿美眉，极致诱惑 [20P]
紧致蓝牛美眉乘车回家[10P]
搭讪清纯可爱漂亮美眉试穿丝袜[12P]
蓝色制服配上性感丝袜美腿实在太完美了！[10P]
一身黑装少女[15P]
街头撞上这小极品，特爽，果断留下[12P]
公交车上的香艳场景，拍到的“白馒头”[20P]
丰满的黑色连衣长裙美眉[14P]
街拍的吊带上衣短裤美腿妹子好长好长的大白腿[10P]
一群美女一起上，你受得了诱惑吗？[11P]
包臀黑色裙白高跟气质性感美少妇[20P]
跟拍超短皮裤黑丝美腿MM[10P]
丰满圆臀热裤妹子，紧绷迷人 [9P]
海边比基尼女孩[11P]
极品黑丝短裙美少妇[9P]
丰满圆润，紧身包臀裙[10P]
粉色包臀和黑色包臀超高跟的两个美女你喜欢哪个 [11P]
白裙少妇 [11P]


In [7]:
import requests
import os
import time
import re
from bs4 import BeautifulSoup

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
url = 'http://www.52yuanwei.live/thread-157828-1-1.html'
resp = requests.get(url, headers=headers)
#resp.encoding = 'content-type'
soup = BeautifulSoup(resp.text, 'lxml')



print(soup)
t1 = soup.find_all('ignore_js_op')
print(t1)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>提示信息 - 我爱原味网</title>
<link href="http://www.52yuanwei.live/thread-157828-1-1.html" rel="canonical"/>
<meta content="" name="keywords"/>
<meta content=",我爱原味网" name="description"/>
<meta content="Discuz! X3.4" name="generator"/>
<meta content="Discuz! Team and Comsenz UI Team" name="author"/>
<meta content="2001-2017 Comsenz Inc." name="copyright"/>
<meta content="True" name="MSSmartTagsPreventParsing"/>
<meta content="Yes" http-equiv="MSThemeCompatible"/>
<base href="http://www.52yuanwei.live/"/><link href="data/cache/style_2_common.css?xQK" rel="stylesheet" type="text/css"/><link href="data/cache/style_2_forum_viewthread.css?xQK" rel="stylesheet" type="text/css"/><script type="text/javascript">var STYLEID = '2', STATICURL = 'static/', IMGDI